# Demo for showcasing ETL pipeline for Cassandra

In [1]:
# Importing Python packages 

import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
# Geting Current Working directory and attaching event_data subfolder in the path

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

In [3]:
# Processing all the files to create a new single data file that can be processed and inserted into tables for querying

full_data_rows_list = [] 
    
# Loop for every file in the directory 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
         # extracting each data record one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Creating a new single file from the collated values of all files
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''): # Skipping records with no data in first field (Artist Name) 
            continue
        # Writing useful fields to a new file    
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16])) 

In [4]:
# Creating a connection to a Cassandra instance 
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [5]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS ankit 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

In [6]:
# Setting KEYSPACE
try:
    session.set_keyspace('ankit')
except Exception as e:
    print(e)

### Next three Blocks are detailed for providing a solution to following type of queries :-
### Give me the artist name, song title and song's length in the music app history that was heard during session Id = "session id" and item In Session  = "Item in session Id"

In [7]:
# Since mutiple songs can be played in a session hence that can't be single unique column hence a numbered column (Item in Session) is added for uniqueness. 
# Given the nature of query, Primary key for the above query is defined as Composite primary key on Session_Id and item_in_session column.
# Above composite Primary key give uniquesness to data hence a perfect candidate for Primary key in this scenario.
# Composite Primary Key - session_id, item_in_session - Providing uniqueness to each record.
# Partition Key - session_id - Session Id will distribute the data evenly across nodes. This might not be entirely correct as data in one session caould be 
#                              different than data in another but given the nature of query this is the best bet.  
# Clustering Key - item_in_session - Providing sorting within the Partition. 

query = "create table if not exists session_details"
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [8]:
# Inserting the relevant fields from new single file into the session_details table.
file_name = 'event_datafile_new.csv'

with open(file_name, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skipping header
    for line in csvreader:
        # Inserting only relevant columns for the specific query.
        query = "insert into session_details(session_id,item_in_session, artist_name, song_title, song_length)"
        query = query + "values (%s,%s,%s,%s,%s)"
        # Inserting record in table after appropriate data type conversion
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [9]:
# Selecting data from table to for outputting the desired result.
query = "select artist_name, song_title, song_length from session_details where session_id=139 and item_in_session=1"

try:
    output_data = session.execute(query)
except Exception as e:
    print(e)

# Output for Query 1
print("Details for song that have been played in Session Id = 139 and correspoding to Item in the session = 1 is as follows: ")

for row in output_data:
    print("Artist Name is : " + row.artist_name +", Song is : " + row.song_title + " and Song Length is : " + str(row.song_length))

Details for song that have been played in Session Id = 139 and correspoding to Item in the session = 1 is as follows: 
Artist Name is : Des'ree, Song is : You Gotta Be and Song Length is : 246.30812072753906


### Next three Blocks are detailed for providing a solution to following type of queries :-
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = "Input User Id", sessionid = "Input Session Id"

In [10]:
# A session can have mutiple songs played by a user hence these two alone can't provide uniqueness.  
# session_id, user_id and item_in_session column provide the uniqueness to the record (based on nature of query).
# user_id or session_id could be a candidate for Partition key but looking at nature of data every session will 
# only be associarted with a single user id hence both together will form a better Composite Partition key. However
# since a user can span across multiple session hence session_id would be the first column while user_id would be second. 

# Composite Primary Key - session_id, user_id, item_in_session - Providing uniqueness to each record.
# Composite Partition Key - session_id, user_id  - Providing a good evenly distributed data (based on query).  
# Clustering Key - item_in_session - Providing sorting within the Partition.

query = "create table if not exists user_session_details"

query = query + "(session_id int, user_id int, item_in_session int, first_name text, last_name text, song_title text, \
                  artist_name text, PRIMARY KEY ((session_id, user_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
# Inserting the relevant fields from new sigle file into the session_details table.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Inserting only relevant columns for the specific query.
        query = "insert into user_session_details( session_id, user_id, item_in_session, first_name, last_name, song_title, artist_name)"
        query = query + "values (%s,%s,%s,%s,%s,%s,%s)"
        # Inserting record in table after appropriate data type conversion
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[1],line[4], line[9], line[0]))

In [12]:
# Selecting data from table for outputting the desired result.
query = "select artist_name, song_title, first_name, last_name from user_session_details where session_id = 139 and user_id=8"
try:
    output_data = session.execute(query)
except Exception as e:
    print(e)

# Output for Query 2
print("Details of song that has been played by a user in a session (user id=8 and session_id=139) is as follows: ")

for row in output_data:
    print("User Name is : " + row.first_name+ ' ' + row.last_name+", Artist Name is : " + row.artist_name +" and Song title is : " + row.song_title)

Details of song that has been played by a user in a session (user id=8 and session_id=139) is as follows: 
User Name is : Kaylee Summers, Artist Name is : Des'ree and Song title is : You Gotta Be
User Name is : Kaylee Summers, Artist Name is : Mr Oizo and Song title is : Flat 55
User Name is : Kaylee Summers, Artist Name is : Tamba Trio and Song title is : Quem Quiser Encontrar O Amor
User Name is : Kaylee Summers, Artist Name is : The Mars Volta and Song title is : Eriatarka
User Name is : Kaylee Summers, Artist Name is : Infected Mushroom and Song title is : Becoming Insane
User Name is : Kaylee Summers, Artist Name is : Blue October / Imogen Heap and Song title is : Congratulations
User Name is : Kaylee Summers, Artist Name is : Girl Talk and Song title is : Once again


### Next three Blocks are detailed for providing a solution to following type of queries :-
### Give me every user name (first and last) in my music app history who listened to the song =" Input Song Name"

In [13]:
# A song can be played by mutiple users across session. Hence Song itself is not enough to provide uniqueness to record.
# Hence song is paired up with user_id to provide uniqueness.  

# Composite Primary Key - song_title, user_id - Providing uniqueness to each record.
# Partition Key - song_title  - Providing a good evenly distributed data (based on query).  
# Clustering Key - user_id - Providing sorting within the Partition.

query = "create table if not exists user_song_details"

query = query + "(song_title text,user_id int, sesion_id int, item_in_session int, first_name text, last_name text,PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)        

In [15]:
# Inserting the relevant fields from new sigle file into the session_details table.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Inserting only relevant columns for the specific query.
        query = "insert into user_song_details(song_title, user_id, first_name, last_name)"
        query = query + "values (%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
# Selecting data from table for outputting the desired result.

query = "select first_name,last_name from user_song_details where song_title='Fireflies'"

try:
    output_data = session.execute(query)
except Exception as e:
    print(e)
    
# Output for Query 3
print("Details of users who has played the song ('Fireflies') are as follows:")

for row in output_data:
    print ("User name is : " + row.first_name + ' ' + row.last_name)

Details of users who has played the song ('Fireflies') are as follows:
User name is : Chloe Cuevas
User name is : Jayden Fox


In [17]:
# Dropping the tables
# Selecting data from table for outputting the desired result.

query = "drop table if exists session_details"

try:
    output_data = session.execute(query)
except Exception as e:
    print(e)

query = "drop table if exists user_session_details"

try:
    output_data = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists user_song_details"

try:
    output_data = session.execute(query)
except Exception as e:
    print(e)

In [18]:
# Closing session and cluster

session.shutdown()
cluster.shutdown()